In [22]:
from contourpy.array import concat_points_or_none

from common.inputreader import InputReader, PuzzleWrapper
from common.matrix import MatrixNavigator

puzzle = PuzzleWrapper(year=2024, day=int("12"))

puzzle.header()
# example = get_code_block(puzzle, 5)

# Garden Groups

[Open Website](https://adventofcode.com/2024/day/12)

In [23]:
# helper functions
def domain_from_input(input: InputReader) -> list:
    lines = input.matrix()

    return lines


test_input = domain_from_input(puzzle.get_code_block(2))
test_input.print()

RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE


In [24]:
from common.matrix import Direction, MatrixNavigator


class Plot:
    def __init__(self, letter: str):
        self.letter = letter
        self.positions = []

    def add(self, pos):
        self.positions.append(pos)

    def total_positions(self):
        return len(self.positions)

    def total_perimeter(self):
        # iterate over positions
        perimeter = 0
        for x, y in self.positions:
            for direction in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                x1 = x + direction[0]
                y1 = y + direction[1]
                if (x1, y1) not in self.positions:
                    perimeter += 1
        return perimeter

    def __str__(self):
        return f"Plot: {self.letter} - {self.positions}"


# test case (part 1)
def part_1(reader: InputReader, debug: bool) -> int:
    matrix = domain_from_input(reader)

    # find all the letters
    letters = set()
    for x, y, letter in matrix:
        letters.add(letter)

    directions = [
        Direction.UP,
        Direction.DOWN,
        Direction.LEFT,
        Direction.RIGHT
    ]

    def find_plot(initial_start: MatrixNavigator) -> Plot:
        plot = Plot(initial_start.get_value())
        plot.add(initial_start.current_position)

        candidates = [initial_start.current_position]

        # loop through candidates
        while candidates:
            start = candidates.pop()
            start_pointer = MatrixNavigator(matrix, start[0], start[1])
            for direction in directions:
                pointer = start_pointer.copy()
                ok = pointer.move(direction)
                if ok and pointer.get_value() == plot.letter:
                    if pointer.current_position not in plot.positions:
                        plot.add(pointer.current_position)
                        candidates.append(pointer.current_position)

        return plot

    history = set()
    plots = []

    for x, y, letter in matrix:
        pointer = MatrixNavigator(matrix, x, y)
        if pointer.current_position in history:
            continue

        plot = find_plot(pointer)
        plots.append(plot)
        for position in plot.positions:
            history.add(position)

    total = 0

    for plot in plots:
        if debug:
            print(plot)
            print(f"total positions: {plot.total_positions()} total perimeter: {plot.total_perimeter()}")
        total += plot.total_positions() * plot.total_perimeter()

    if debug:
        matrix.print()
    return total


result = part_1(puzzle.get_code_block(2), True)
display(result)
assert result == 1930

Plot: R - [(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1), (3, 0), (3, 1), (3, 2), (2, 2), (4, 2), (2, 3)]
total positions: 12 total perimeter: 18
Plot: I - [(4, 0), (4, 1), (5, 0), (5, 1)]
total positions: 4 total perimeter: 8
Plot: C - [(6, 0), (6, 1), (7, 0), (7, 1), (8, 1), (6, 2), (5, 2), (5, 3), (4, 3), (4, 4), (3, 3), (4, 5), (5, 5), (5, 6)]
total positions: 14 total perimeter: 28
Plot: F - [(8, 0), (9, 0), (9, 1), (9, 2), (9, 3), (8, 2), (8, 3), (7, 2), (7, 3), (8, 4)]
total positions: 10 total perimeter: 18
Plot: V - [(0, 2), (0, 3), (1, 2), (1, 3), (1, 4), (1, 5), (0, 4), (2, 4), (3, 4), (3, 5), (0, 5), (0, 6), (1, 6)]
total positions: 13 total perimeter: 20
Plot: J - [(6, 3), (6, 4), (6, 5), (5, 4), (6, 6), (7, 5), (7, 6), (7, 7), (6, 7), (6, 8), (6, 9)]
total positions: 11 total perimeter: 20
Plot: C - [(7, 4)]
total positions: 1 total perimeter: 4
Plot: E - [(9, 4), (9, 5), (9, 6), (8, 5), (8, 6), (8, 7), (8, 8), (9, 7), (9, 8), (9, 9), (8, 9), (7, 9), (7, 8)]
total positi

1930

In [25]:
# real case (part 1)
result = part_1(puzzle.input(), False)
display(result)
assert 1319878 == result

1319878

In [28]:
# test case (part 2)
class Plot2:
    def __init__(self, letter: str):
        self.letter = letter
        self.positions = []

    def add(self, pos):
        self.positions.append(pos)

    def total_positions(self):
        return len(self.positions)

    def get_edges(self):
        edges = []
        for x, y in self.positions:
            neighbors = self.find_neighbors(x, y, self.positions)
            if len(neighbors) < 4:
                edges.append((x, y))
        return edges

    @staticmethod
    def find_neighbors(x, y, positions):
        neighbors = []
        for direction in [(-1, 0), (0, 1), (1, 0), (0, -1)]:
            x1 = x + direction[0]
            y1 = y + direction[1]
            if (x1, y1) in positions:
                neighbors.append((x1, y1))
        return neighbors

    def total_sides(self):
        edges = 0
        points = self.positions
        min_x = min(x for x, _ in points)
        max_x = max(x for x, _ in points)
        min_y = min(y for _, y in points)
        max_y = max(y for _, y in points)

        # create iterators
        range_y = range(min_y, max_y + 1)
        range_x = range(min_x, max_x + 1)

        for y in range_y:
            top_was_edge = False
            bottom_was_edge = False
            for x in range_x:
                top_is_edge = False
                bottom_is_edge = False

                if (x, y) in points:
                    neighbors = self.find_neighbors(x, y, self.positions)
                    top_is_edge = (x, y - 1) not in neighbors
                    bottom_is_edge = (x, y + 1) not in neighbors

                    if top_is_edge and not top_was_edge:
                        edges += 1
                    if bottom_is_edge and not bottom_was_edge:
                        edges += 1

                top_was_edge = top_is_edge
                bottom_was_edge = bottom_is_edge

        for x in range_x:
            left_was_edge = False
            right_was_edge = False
            for y in range_y:
                left_is_edge = False
                right_is_edge = False

                if (x, y) in points:
                    neighbors = self.find_neighbors(x, y, self.positions)
                    left_is_edge = (x - 1, y) not in neighbors
                    right_is_edge = (x + 1, y) not in neighbors

                    if left_is_edge and not left_was_edge:
                        edges += 1
                    if right_is_edge and not right_was_edge:
                        edges += 1

                left_was_edge = left_is_edge
                right_was_edge = right_is_edge

        return edges

    def __str__(self):
        return f"Plot: {self.letter} - {self.positions}"


def part_2(reader: InputReader, debug: bool) -> int:
    matrix = domain_from_input(reader)

    # find all the letters
    letters = set()
    for x, y, letter in matrix:
        letters.add(letter)

    directions = [
        Direction.UP,
        Direction.DOWN,
        Direction.LEFT,
        Direction.RIGHT
    ]

    def find_plot(initial_start: MatrixNavigator) -> Plot2:
        plot = Plot2(initial_start.get_value())
        plot.add(initial_start.current_position)

        candidates = [initial_start.current_position]

        # loop through candidates
        while candidates:
            start = candidates.pop()
            start_pointer = MatrixNavigator(matrix, start[0], start[1])
            for direction in directions:
                pointer = start_pointer.copy()
                ok = pointer.move(direction)
                if ok and pointer.get_value() == plot.letter:
                    if pointer.current_position not in plot.positions:
                        plot.add(pointer.current_position)
                        candidates.append(pointer.current_position)

        return plot

    history = set()
    plots = []

    for x, y, letter in matrix:
        pointer = MatrixNavigator(matrix, x, y)
        if pointer.current_position in history:
            continue

        plot = find_plot(pointer)
        plots.append(plot)
        for position in plot.positions:
            history.add(position)

    total = 0

    for plot in plots:
        if debug:
            print(plot)
            print(f"total positions: {plot.total_positions()} total perimeter: {plot.total_sides()}")
        total += plot.total_positions() * plot.total_sides()

    if debug:
        matrix.print()
    return total


result = part_2(puzzle.get_code_block(2), True)
display(result)
assert result == 1206

Plot: R - [(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1), (3, 0), (3, 1), (3, 2), (2, 2), (4, 2), (2, 3)]
total positions: 12 total perimeter: 10
Plot: I - [(4, 0), (4, 1), (5, 0), (5, 1)]
total positions: 4 total perimeter: 4
Plot: C - [(6, 0), (6, 1), (7, 0), (7, 1), (8, 1), (6, 2), (5, 2), (5, 3), (4, 3), (4, 4), (3, 3), (4, 5), (5, 5), (5, 6)]
total positions: 14 total perimeter: 22
Plot: F - [(8, 0), (9, 0), (9, 1), (9, 2), (9, 3), (8, 2), (8, 3), (7, 2), (7, 3), (8, 4)]
total positions: 10 total perimeter: 12
Plot: V - [(0, 2), (0, 3), (1, 2), (1, 3), (1, 4), (1, 5), (0, 4), (2, 4), (3, 4), (3, 5), (0, 5), (0, 6), (1, 6)]
total positions: 13 total perimeter: 10
Plot: J - [(6, 3), (6, 4), (6, 5), (5, 4), (6, 6), (7, 5), (7, 6), (7, 7), (6, 7), (6, 8), (6, 9)]
total positions: 11 total perimeter: 12
Plot: C - [(7, 4)]
total positions: 1 total perimeter: 4
Plot: E - [(9, 4), (9, 5), (9, 6), (8, 5), (8, 6), (8, 7), (8, 8), (9, 7), (9, 8), (9, 9), (8, 9), (7, 9), (7, 8)]
total positi

1206

In [29]:
# real case (part 2)
result = part_2(puzzle.input(), False)
display(result)
assert result == 784982

784982

In [21]:
# print easters eggs
puzzle.print_easter_eggs()

## Easter Eggs

<span title='I originally wanted to title this puzzle "Fencepost Problem", but I was afraid someone would then try to count fenceposts by mistake and experience a fencepost problem.'>fences</span> (I originally wanted to title this puzzle "Fencepost Problem", but I was afraid someone would then try to count fenceposts by mistake and experience a fencepost problem.)